# 1. Bussiness Understanding

2. Data Understanding - Data Collection

## Importing Libraries

In [184]:
#import semua library yang diperlukan

import pandas as pd, numpy as np, matplotlib.pyplot as plt
import json, tweepy, requests, re
import seaborn as sns
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import gensim
from gensim import corpora
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from numpy import array
import numpy as np
import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [166]:
import json, re, string
import pandas as pd
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory as SF
import nltk

## Data Collection

In [167]:
with open("token.json") as f:
  tokens = json.load(f)

bearer_token = tokens['bearer_token']
api_key = tokens['api_key']
api_key_secret = tokens['api_key_secret']
access_token = tokens['access_token']
access_token_secret = tokens['access_token_secret']

In [169]:
api = tweepy.Client(bearer_token=bearer_token, wait_on_rate_limit=True)

In [172]:
query = "pemerintah jokowi -is:retweet lang:id"
start_time = '2022-06-19T00:00:00Z'
end_time = '2022-06-22T00:00:00Z'


response = tweepy.Paginator(api.search_recent_tweets,
                          query = query,
                          start_time = start_time,
                          end_time = end_time,
                          max_results=100
                          ).flatten()

In [180]:
tweets = [tweet.text.strip() for tweet in response]
df_tweets = pd.DataFrame(tweets, columns=["tweets"])
df_tweets

,tweets


In [181]:
column_list=['tweets']
df_tweets.to_csv('pemerintahjokowi_tweet.csv', columns = column_list, index=False)

In [182]:
kompas = requests.get('https://sorotpolitik.kompas.com/memilih-pemimpin-negeri/2')
beautify = BeautifulSoup(kompas.content,'html5lib')

news = beautify.find_all('div', {'class','article__list clearfix'})
titles = []

for i in news:
  news_title = i.find('div', {'class','article__list__title'}).get_text()

  titles.append({
    'tweets': news_title,
  })

df = pd.DataFrame(titles)
df.to_csv('news.csv', index=False)

## Data Preparation

In [183]:
df_tweets = pd.read_csv('pemerintahjokowi_tweet.csv')
df_tweets.head()

,tweets


In [159]:
df_news = pd.read_csv('news.csv')
df_news.head()

,tweets
0,\n Sinergi Pemerintah dan Pengusaha Jadi Ku...
1,"\n Keputusan Politik Berani Jokowi, Antar J..."
2,\n Alasan Pemerintah Jokowi-JK Fokus Bangun...
3,\n Usaha Pemerintah untuk Jadikan Profesi P...
4,"\n Punya Potensi Besar, Industri Keramik Ja..."


In [160]:
df_news = df_news.replace('\n','', regex=True)
df_news.head()

,tweets
0,Sinergi Pemerintah dan Pengusaha Jadi Kunc...
1,"Keputusan Politik Berani Jokowi, Antar Jak..."
2,Alasan Pemerintah Jokowi-JK Fokus Bangun I...
3,Usaha Pemerintah untuk Jadikan Profesi Pet...
4,"Punya Potensi Besar, Industri Keramik Jadi..."


In [163]:
data_join = pd.merge(df_tweets, df_news,
                on = 'tweets',
                how = 'outer')

In [165]:
df_tweets['tweets'] = data_join
df_tweets

,tweets
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
48316,NaN
48317,NaN
48318,NaN
48319,NaN


## Data Cleaning

4. Data Understanding - Data Preprocessing

In [154]:
def case_folding():
    for data in df_tweets['tweets']:
        low_cases =  df_tweets['tweets'].str.lower()
        remove_number = low_cases.str.replace('\d+', '', regex=True)
        remove_punctuation = remove_number.str.replace(r'[^\w\s]+', '', regex=True)
        remove_link = remove_punctuation.replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)
        remove_whitespace = remove_link.str.strip()
        return remove_whitespace

result = case_folding()
df_tweets['clean'] = result
df_tweets

,tweets,clean
0,@triatmodjosast1 @jokowi ini baru dengan simbo...,triatmodjosast jokowi ini baru dengan simbok g...
1,Persmian SDGs Center\n@jokowi \n@halimiskandar...,persmian sdgs center\njokowi \nhalimiskandarnu...
2,@libertees_id jadilah seperti jokowi taapii em...,libertees_id jadilah seperti jokowi taapii emg...
3,@jokowi Matur suwun,jokowi matur suwun
4,@kurawa Saya suka kepribadian bapak Jokowi yg ...,kurawa saya suka kepribadian bapak jokowi yg l...
5,@D_N4bil5 @RadjaRasta_ @jokowi @newIding30 @Tr...,d_nbil radjarasta_ jokowi newiding tritung pra...
6,@harisubagya @cybsquad_ Mega awet nom 🤣 jokowi...,harisubagya cybsquad_ mega awet nom jokowi ae...
7,@Miduk17 @PDI_Perjuangan @jokowi Terlalu meren...,miduk pdi_perjuangan jokowi terlalu merendah j...
8,Peresmian SDGs Desa Center : Kemendes dan Keme...,peresmian sdgs desa center kemendes dan kemen...
9,@porenges666 @prasariv @masTenky Kalo dihadapa...,porenges prasariv mastenky kalo dihadapan kaes...


In [155]:
#buat sebuah fungsi untuk menghilangkan stopwords
'''
menghilangkan stopword bahasa indonesia + kata bandung dan kota
'''
def stop_word():
    new_stopwords = ['bandung', 'kota']
    stop_words = stopwords.words('indonesian')
    stop_words.extend(new_stopwords)
    word_tokens = df['clean'].apply(word_tokenize)
    filtered_text = word_tokens.apply(lambda words: [word for word in words if word not in stop_words])
    response = filtered_text.apply(lambda x: ' '.join([word for word in x if word not in stop_words]))
    
    return response

result = stop_word()
df_tweets['clean'] = result
df_tweets

,tweets,clean
0,@triatmodjosast1 @jokowi ini baru dengan simbo...,triatmodjosast jokowi simbok gimana jipeng pen...
1,Persmian SDGs Center\n@jokowi \n@halimiskandar...,persmian sdgs center jokowi halimiskandarnu ke...
2,@libertees_id jadilah seperti jokowi taapii em...,libertees_id jokowi taapii emg pas ak ultah g ...
3,@jokowi Matur suwun,jokowi matur suwun
4,@kurawa Saya suka kepribadian bapak Jokowi yg ...,kurawa suka kepribadian jokowi yg legowo video...
5,@D_N4bil5 @RadjaRasta_ @jokowi @newIding30 @Tr...,d_nbil radjarasta_ jokowi newiding tritung pra...
6,@harisubagya @cybsquad_ Mega awet nom 🤣 jokowi...,harisubagya cybsquad_ mega awet nom jokowi ae ...
7,@Miduk17 @PDI_Perjuangan @jokowi Terlalu meren...,miduk pdi_perjuangan jokowi merendah jdi bikn ...
8,Peresmian SDGs Desa Center : Kemendes dan Keme...,peresmian sdgs desa center kemendes kemenkeu r...
9,@porenges666 @prasariv @masTenky Kalo dihadapa...,porenges prasariv mastenky kalo dihadapan kaes...


In [157]:
df_tweets.dropna(subset = ["clean"], inplace=False)
df_tweets

,tweets,clean
0,@triatmodjosast1 @jokowi ini baru dengan simbo...,triatmodjosast jokowi simbok gimana jipeng pen...
1,Persmian SDGs Center\n@jokowi \n@halimiskandar...,persmian sdgs center jokowi halimiskandarnu ke...
2,@libertees_id jadilah seperti jokowi taapii em...,libertees_id jokowi taapii emg pas ak ultah g ...
3,@jokowi Matur suwun,jokowi matur suwun
4,@kurawa Saya suka kepribadian bapak Jokowi yg ...,kurawa suka kepribadian jokowi yg legowo video...
5,@D_N4bil5 @RadjaRasta_ @jokowi @newIding30 @Tr...,d_nbil radjarasta_ jokowi newiding tritung pra...
6,@harisubagya @cybsquad_ Mega awet nom 🤣 jokowi...,harisubagya cybsquad_ mega awet nom jokowi ae ...
7,@Miduk17 @PDI_Perjuangan @jokowi Terlalu meren...,miduk pdi_perjuangan jokowi merendah jdi bikn ...
8,Peresmian SDGs Desa Center : Kemendes dan Keme...,peresmian sdgs desa center kemendes kemenkeu r...
9,@porenges666 @prasariv @masTenky Kalo dihadapa...,porenges prasariv mastenky kalo dihadapan kaes...


In [104]:
def stopword_cleaner(data):
    sw_indonesia = stopwords.words("indonesian")
    data  = [word for word in data if word not in sw_indonesia]
    data = ' '.join(data)
    return data

In [113]:
for index in range (len(df_tweets['clean']))
  df_tweets['clean'].iloc[index] = stopword_cleaner(df_tweets['clean'].iloc[index])

SyntaxError: invalid syntax (1981478927.py, line 1)

In [90]:
vectorizer = TfidfVectorizer (max_features=2500)
model_g = GaussianNB()

In [93]:
v_data = vectorizer.fit_transform(df_tweets['tweets']).toarray()
vectorizer.fit_transform(df_tweets['tweets'])

<48321x2500 sparse matrix of type '<class 'numpy.float64'>'
	with 407362 stored elements in Compressed Sparse Row format>

5. Data Processing - Modeling

6. Data Processing - Evaluation

7. Data Processing - Prediction